# Year to year ERA prediction

This notebook analyzes which statistics best predict a pitchers Earned Run Average, (ERA) from one year to the next

In [203]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import sklearn
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from xgboost import XGBRegressor

%matplotlib inline

We first import select pitching data for qualified starting pitchers from www.fangraphs.com in the date range 2002-2017, for an explanation of each statistic see https://www.fangraphs.com/library/pitching/ .

In [243]:
stats=pd.read_csv('/home/bradley/brack228.github.io/pitching_all.csv')

In [244]:
stats.shape

(2547, 43)

One sees that there is a column for each statistic and a row for each pitcher for each season

In [245]:
stats.columns

Index(['Season', 'Name', 'Team', 'Age', 'ERA', 'K/9', 'BB/9', 'K/BB', 'H/9',
       'HR/9', 'AVG', 'WHIP', 'BABIP', 'LOB%', 'GB/FB', 'LD%', 'GB%', 'FB%',
       'IFFB%', 'HR/FB', 'IFH%', 'BUH%', 'O-Swing%', 'Z-Swing%', 'Swing%',
       'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%',
       'K%', 'BB%', 'SIERA', 'RS/9', 'Pull%', 'Cent%', 'Oppo%', 'Soft%',
       'Med%', 'Hard%', 'xFIP', 'playerid'],
      dtype='object')

The included stats are listed above, I have only included rate stats because counting statistics will skew predicitions in favor of pitchers who have pitched more innings in a given season.

Below I reformat the data values in each column so that they are all floats and are suitable for analysis

In [246]:
for col in stats.columns:
    if type(stats[col][1])==str:
        stats[col]=stats[col].str.replace('%','',regex=False)
 

In [247]:
numbers=['Season', 'Age', 'ERA', 'K/9', 'BB/9', 'K/BB', 'H/9',
       'HR/9', 'AVG', 'WHIP', 'BABIP', 'LOB%', 'GB/FB', 'LD%', 'GB%', 'FB%',
       'IFFB%', 'HR/FB', 'IFH%', 'BUH%', 'O-Swing%', 'Z-Swing%', 'Swing%',
       'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%',
       'K%', 'BB%', 'SIERA', 'RS/9', 'Pull%', 'Cent%', 'Oppo%', 'Soft%',
       'Med%', 'Hard%', 'xFIP', 'playerid']
for col in numbers:
    stats[col]=stats[col].astype(float)

In the next three cells I create a new dataframe from the original that includes a column for the given pitchers ERA the following season, and rename the columns accordingly

In [8]:
np.sort(stats.Season.unique())

array([2002., 2003., 2004., 2005., 2006., 2007., 2008., 2009., 2010.,
       2011., 2012., 2013., 2014., 2015., 2016., 2017., 2018., 2019.])

In [61]:
[i for i in range(2002,2019)]

[2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018]

In [248]:
predseasons=['stats'+str(i)+str(i+1) for i in range(2002,2019)]
i=2002
for season in predseasons:
  
    globals()[season] = pd.merge(stats[stats['Season']==i],stats[stats['Season']==(i+1)][['Name','ERA']],on="Name")
    i+=1
   

In [249]:
statsrel=pd.concat([globals()[season] for season in predseasons])

In [250]:
statsrel=statsrel.rename(columns={'ERA_x':'ERA_current_year','ERA_y':'ERA_next_year'})

In [251]:
#statsrel=statsrel[statsrel.Season>=2015]
#statsrel['Name']=' '+statsrel['Name']
statsrel=statsrel.reset_index(drop=True)

In [301]:
cast=pd.read_csv('statcast_pitching.csv')

In [181]:
cast['Name']=cast[' first_name']+' '+cast['last_name']

In [182]:
nums=['year', 'slg_percent', 'on_base_percent',
       'on_base_plus_slg', 'isolated_power', 'exit_velocity_avg',
       'launch_angle_avg', 'sweet_spot_percent', 'barrel_batted_rate',
       'solidcontact_percent', 'flareburner_percent', 'poorlyunder_percent',
       'poorlytopped_percent', 'poorlyweak_percent', 'hard_hit_percent',
       'z_swing_percent', 'z_swing_miss_percent', 'oz_swing_percent',
       'oz_swing_miss_percent', 'oz_contact_percent', 'out_zone_percent',
       'meatball_swing_percent', 'meatball_percent', 'iz_contact_percent',
       'in_zone_percent', 'edge_percent', 'whiff_percent', 'swing_percent',
       'pull_percent', 'straightaway_percent', 'opposite_percent',
       'f_strike_percent', 'groundballs_percent', 'flyballs_percent',
       'linedrives_percent', 'popups_percent']
for col in nums:
    cast[col]=cast[col].astype(float)

In [183]:
cast.rename(columns={'year':'Season'}, inplace=True)

In [184]:
cols=['Season', 'slg_percent', 'on_base_percent',
       'on_base_plus_slg', 'isolated_power', 'exit_velocity_avg',
       'launch_angle_avg', 'sweet_spot_percent', 'barrel_batted_rate',
       'solidcontact_percent', 'flareburner_percent', 'poorlyunder_percent',
       'poorlytopped_percent', 'poorlyweak_percent', 'hard_hit_percent',
       'z_swing_percent', 'z_swing_miss_percent', 'oz_swing_percent',
       'oz_swing_miss_percent', 'oz_contact_percent', 'out_zone_percent',
       'meatball_swing_percent', 'meatball_percent', 'iz_contact_percent',
       'in_zone_percent', 'edge_percent', 'whiff_percent', 'swing_percent',
       'pull_percent', 'straightaway_percent', 'opposite_percent',
       'f_strike_percent', 'groundballs_percent', 'flyballs_percent',
       'linedrives_percent', 'popups_percent','Name']
cast=cast[cols]

In [185]:
all_stats=pd.merge(statsrel,cast,on=['Name','Season'])

In [213]:
statsrel=all_stats

"features" is a list of statistics that we will use to try to predict a pitchers ERA.

"featureswithestimators" also includes the ERA estimators xFIP (eXpected Fielding Independent Pitching) and SIERA (Skill Interactive ERA), these stats use strikeout, walk and fly ball rates to say what a pitchers ERA 'should' be by removing factors such as team defense that are out of the pitchers control. For more information on these stats see https://www.fangraphs.com/library/pitching/. 

In [214]:
statsrel.columns

Index(['Season', 'Name', 'Team', 'Age', 'ERA_current_year', 'K/9', 'BB/9',
       'K/BB', 'H/9', 'HR/9', 'AVG', 'WHIP', 'BABIP', 'LOB%', 'GB/FB', 'LD%',
       'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH%', 'BUH%', 'O-Swing%', 'Z-Swing%',
       'Swing%', 'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%',
       'SwStr%', 'K%', 'BB%', 'SIERA', 'RS/9', 'Pull%', 'Cent%', 'Oppo%',
       'Soft%', 'Med%', 'Hard%', 'xFIP', 'playerid', 'ERA_next_year',
       'slg_percent', 'on_base_percent', 'on_base_plus_slg', 'isolated_power',
       'exit_velocity_avg', 'launch_angle_avg', 'sweet_spot_percent',
       'barrel_batted_rate', 'solidcontact_percent', 'flareburner_percent',
       'poorlyunder_percent', 'poorlytopped_percent', 'poorlyweak_percent',
       'hard_hit_percent', 'z_swing_percent', 'z_swing_miss_percent',
       'oz_swing_percent', 'oz_swing_miss_percent', 'oz_contact_percent',
       'out_zone_percent', 'meatball_swing_percent', 'meatball_percent',
       'iz_contact_percent'

In [252]:
features=['Age', 'ERA_current_year', 'K/9', 'BB/9',
       'K/BB', 'H/9', 'HR/9', 'AVG', 'WHIP', 'BABIP', 'LOB%', 'GB/FB', 'LD%',
       'GB%', 'FB%', 'IFFB%', 'HR/FB','O-Swing%', 'Z-Swing%',
       'Swing%', 'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%',
       'SwStr%', 'K%', 'BB%','Pull%', 'Cent%', 'Oppo%',
       'Soft%', 'Med%', 'Hard%']


In [253]:
featureswithestimators=['Age', 'ERA_current_year', 'K/9', 'BB/9',
       'K/BB', 'H/9', 'HR/9', 'AVG', 'WHIP', 'BABIP', 'LOB%', 'GB/FB', 'LD%',
       'GB%', 'FB%', 'IFFB%', 'HR/FB', 'O-Swing%', 'Z-Swing%',
       'Swing%', 'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%',
       'SwStr%', 'K%', 'BB%','Pull%', 'Cent%', 'Oppo%',
       'Soft%', 'Med%', 'Hard%','xFIP','SIERA']

First lets explore which stats are most correlated with a pitcher's ERA during the same year

In [215]:
corrdict1={col:np.abs(statsrel[col].corr(statsrel['ERA_current_year'])) for col in featureswithestimators}



    
Corr=pd.DataFrame.from_dict(corrdict1, orient='index')
Corr.sort_values(by=0,ascending=False)

,0
ERA_current_year,1.000000
LOB%,0.759095
WHIP,0.751698
H/9,0.697129
AVG,0.674024
HR/9,0.527633
xFIP,0.521690
SIERA,0.516209
BABIP,0.487042
HR/FB,0.398085


We see that WHIP and hits per nine innings have strong correlations to ERA, placing just above xFIP and SIERA as the most strongly correlated variables. Now lets take a look at how these stats are correlated to a pitcher's ERA the following season.

In [97]:
statsrel.columns

Index(['Season', 'Name', 'Team', 'Age', 'ERA_current_year', 'K/9', 'BB/9',
       'K/BB', 'H/9', 'HR/9', 'AVG', 'WHIP', 'BABIP', 'LOB%', 'GB/FB', 'LD%',
       'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH%', 'BUH%', 'O-Swing%', 'Z-Swing%',
       'Swing%', 'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%',
       'SwStr%', 'K%', 'BB%', 'SIERA', 'RS/9', 'Pull%', 'Cent%', 'Oppo%',
       'Soft%', 'Med%', 'Hard%', 'xFIP', 'playerid', 'ERA_next_year'],
      dtype='object')

In [121]:
corrdict2={col:np.abs(statsrel[col].corr(statsrel['ERA_next_year'])) for col in featureswithestimators}



    
Corr=pd.DataFrame.from_dict(corrdict2, orient='index')
Corr.sort_values(by=0,ascending=False)'IFH%', 'BUH%'

,0
SIERA,0.268482
K%,0.261745
xFIP,0.251529
K/9,0.242782
AVG,0.239766
H/9,0.230923
WHIP,0.207988
Contact%,0.193359
SwStr%,0.182884
ERA_current_year,0.178364


   Here we see a very different picture, SIERA and xFIP have moved to the front of the pack, justifying their use as truer measures of a pitchers talent than ERA alone. We see that WHIP, and H/9 are now only the 8th and 10th most correlated stats respectively. 
    
   Besides xFIP and SIERA, it seems that the best raw stats to use to predict a pitchers ERA are the ones involving strikeouts, (K%, K/9,K/BB, SwStr%). Its no surprise then that strikeouts are a major component of both the SIERA and xFIP estimators.
    
   Also of note is that a pitchers walk rate, (BB%) has a very low correlation to his ERA the next year. It seems that walks do not hurt a pitcher very much as long as he maintains high strikeout totals.
     

Finally, we try to use a combination of the raw features in a linear regressor to predict a pitchers ERA the following year.

First we build our design matrix and define our feature scaler.

In [254]:
scaler=StandardScaler()

In [255]:
scaler=StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(statsrel[featureswithestimators],statsrel['ERA_next_year'])
X_train=X_train[features]
X_train.shape


(1229, 34)

In [256]:
X=statsrel[features]
y=statsrel['ERA_next_year']

 Next we shuffle the order of the data so that the initial ordering does not bias the model.

In [257]:
from sklearn.utils import shuffle
X_train,y_train=shuffle(X_train,y_train)
X,y=shuffle(X,y)

Now we can use a linear regressor on the data to try and make predictions. We test our model using 5 fold cross validation, and evaluate it using the mean absolute error.

In [258]:
import sklearn
MSEERA = sklearn.metrics.r2_score(y_true = statsrel['ERA_next_year'], y_pred = statsrel.ERA_current_year)
MSEERA

-0.2609413701677288

In [259]:

regressor=LinearRegression()
pipeline=Pipeline([('scaler',MinMaxScaler()),('regressor',regressor)])
E=np.mean(cross_val_score(pipeline,X,y,scoring='r2'))
E
#pipeline.fit(X_train,y_train)
#MAE=sklearn.metrics.mean_absolute_error(y_true = y_test, y_pred = pipeline.predict(X_test[features]))
#MAE


0.17264291245512292

We see that our model is off by an average of about .580 runs in predicting a pitcher's ERA. For comparison, lets look at our errors if we use the pitchers ERA from the previous year as our predicition.

Our regressor fares significantly better, how about if we use SIERA and xFIP?


In [260]:
MAESIERA = sklearn.metrics.r2_score(y_true = statsrel['ERA_next_year'], y_pred =statsrel['SIERA']  )
MAESIERA

0.15601474066311927

In [131]:
MAExFIP = sklearn.metrics.mean_squared_error(y_true = statsrel['ERA_next_year'], y_pred =statsrel['xFIP']  )
np.sqrt(MAExFIP)

1.1985245594612994

Our regressor fares slightly better than xFIP and SIERA, which is no surprise given they both largely use stats from our list of features as inputs.




We can improve our Regression model further by incorporating a regularization scheme, this will help keep the model from overfitiing the data. Lets first try a Ridge regressor, which helps keep the coefficients of our features small. 

In [269]:
from sklearn.linear_model import Ridge

regressor=Ridge(alpha=400)
pipeline=Pipeline([('scaler',scaler),('regressor',regressor)])
E=np.mean(cross_val_score(pipeline,X,y,scoring='r2'))
E
#pipeline.fit(X_train,y_train)
#MAE=sklearn.metrics.mean_absolute_error(y_true = y_test, y_pred = pipeline.predict(X_test[features]))
#MAE



0.18862837876595037

We see a slight improvement in our MAE from earlier (.576 vs .580). Next lets try a Lasso regressor, this will make the algorithm concentrate on a smaller number of features.

In [280]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso

regressor=Lasso(alpha=0.01)
pipeline=Pipeline([('scaler',scaler),('regressor',regressor)])
E=np.mean(cross_val_score(pipeline,X,y,scoring='r2'))
E
#pipeline.fit(X_train,y_train)
#MAE=sklearn.metrics.mean_absolute_error(y_true = y_test, y_pred = pipeline.predict(X_test[features]))
#MAE


0.18984292064784009

In [281]:
np.sqrt(.1898)

0.4356604182158393

In [ ]:
.3 .05

In [102]:
regressor

ElasticNet(alpha=0.1, l1_ratio=0.01)

In [289]:

from sklearn.linear_model import ElasticNet

regressor=ElasticNet(alpha=.1,l1_ratio=.05)
pipeline=Pipeline([('scaler',scaler),('regressor',regressor)])

E=np.mean(cross_val_score(pipeline,X,y,scoring='r2'))
E


0.1892197532307484

In [202]:
import pickle
regressor=ElasticNet(alpha=.1,l1_ratio=.1)
pipeline=Pipeline([('scaler',scaler),('regressor',regressor)])
pipeline.fit(X,y)
pickle.dump(pipeline,open('/home/bradley/baseball/2020_reliever_era_model.pkl','wb'))

In [82]:
pipeline

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor', ElasticNet(alpha=0.3, l1_ratio=0.05))])

In [79]:
pipeline.fit(X,y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor', ElasticNet(alpha=0.3, l1_ratio=0.05))])

In [80]:
pickle

NameError: name 'pickle' is not defined

Finally we try XGBoost, a gradient boosted tree regressor. This solver has many hyperparameters so we employ Randomized grid search to find the optimal combination.

In [240]:
import scipy.stats as st
regressor=XGBRegressor(booster='gbtree',n_estimators=100,max_depth=1,learning_rate=0.05,
                       reg_lambda=0,reg_alpha=10,gamma=0.08,min_child_weight=30)

pipeline=Pipeline([('scaler',scaler),('regressor',regressor)])
parameters = {'regressor__n_estimators': st.randint(50,300),
             'regressor__max_depth':st.randint(1,10),
            'regressor__learning_rate':st.expon(scale=0.05),
             'regressor__reg_alpha':st.expon(scale=5),
              'regressor__reg_lambda':st.expon(scale=.01),
              'regressor__gamma':st.expon(scale=0.08),
             'regressor__min_child_weight':st.randint(5,50)}
model = RandomizedSearchCV(pipeline,parameters,n_iter=100, cv=5, iid=False, n_jobs=-1,refit=True,verbose=5,scoring='r2') 
model.fit(X,y)
#pipeline.fit(X_train,y_train);


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   20.6s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   42.6s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.4min finished
/home/bradley/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('regressor',
                                              XGBRegressor(base_score=None,
                                                           booster='gbtree',
                                                           colsample_bylevel=None,
                                                           colsample_bynode=None,
                                                           colsample_bytree=None,
                                                           gamma=0.08,
                                                           gpu_id=None,
                                                           importance_type='gain',
                                                           interaction_constraints=None,
                                                           learning_rate=0.05,
                                               

In [486]:
MAE=sklearn.metrics.mean_absolute_error(y_true = y_test, y_pred = model.predict(X_test[features]))
MAE

0.37619937786157576

In [88]:
(-1*(model.best_score_))**(.5)

0.8390642177704363

In [241]:
model.best_score_

0.06800259066426939

In [482]:
model.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0.09530873773365127,
                              gpu_id=-1, importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.02985595872045796,
                              max_delta_step=0, max_depth=8,
                              min_child_weight=40, missing=nan,
                              monotone_constraints='()', n_estimators=131,
                              n_jobs=0, num_parallel_tree=1, random_state=0,
                              reg_alpha=0.148849524657472,
                              reg_lambda=0.018742750855552198,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_pa

In [458]:
model.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0.11973106481347351,
                              gpu_id=-1, importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.019398510482160427,
                              max_delta_step=0, max_depth=9,
                              min_child_weight=49, missing=nan,
                              monotone_constraints='()', n_estimators=275,
                              n_jobs=0, num_parallel_tree=1, random_state=0,
                              reg_alpha=5.854796483199712,
                              reg_lambda=0.0016021790992340643,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_

In [493]:
new_mod=model.best_estimator_

In [462]:
new_mod.fit(statsrel[features],statsrel['ERA_next_year'])

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0.004150756294386627,
                              gpu_id=-1, importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.04507246778188143,
                              max_delta_step=0, max_depth=2,
                              min_child_weight=48, missing=nan,
                              monotone_constraints='()', n_estimators=221,
                              n_jobs=0, num_parallel_tree=1, random_state=0,
                              reg_alpha=13.718408488531466,
                              reg_lambda=0.0030009848461364603,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate

In [107]:
regressor=Lasso(alpha=0.001)
pipeline=Pipeline([('scaler',MinMaxScaler()),('regressor',regressor)])
new_mod=pipeline.fit(X,y)

In [108]:
new_mod

Pipeline(steps=[('scaler', MinMaxScaler()), ('regressor', Lasso(alpha=0.001))])

In [109]:
import pickle
pickle.dump(new_mod,open('/home/bradley/baseball/2020_era_model.pkl','wb'))

We print out the best parameters found using grid search and the associated cross-validation score

In [178]:
print('Best Parameters: ' , model.best_params_)
print('\n')
print('Best Score: ' ,  -1*model.best_score_)

Best Parameters:  {'regressor__gamma': 0.054222289156951285, 'regressor__learning_rate': 0.1265594794016572, 'regressor__max_depth': 1, 'regressor__min_child_weight': 6, 'regressor__n_estimators': 54, 'regressor__reg_alpha': 5.450584170340279, 'regressor__reg_lambda': 0.0025626051810866794}


Best Score:  0.5935192649521416


We find XGBoost slightly outperforms the more basic linear regression algorithms (.567 vs .576 mean absolute error)

Finally, lets look at the coefficients of our model to see which stats most strongly influence its prediction. We will do this for our Lasso regression model as it is more straightforward than for the XGBoost model.

In [310]:
regressor=Lasso(alpha=.1)
pipeline=Pipeline([('scaler',scaler),('regressor',regressor)])
pipeline.fit(X,y)

Pipeline(steps=[('scaler', StandardScaler()), ('regressor', Lasso(alpha=0.1))])

In [104]:
regressor.fit(X,y)

ElasticNet(alpha=0.3, l1_ratio=0.05)

In [105]:
np.sort(np.abs(pipeline.named_steps['regressor'].coef_))

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.0001734 , 0.00037407, 0.00080843, 0.00219456, 0.00496123,
       0.00525599, 0.00631908, 0.00963527, 0.0100982 , 0.01074162,
       0.01291765, 0.01510334, 0.01572174, 0.01685014, 0.01921802,
       0.02119584, 0.03183776])

Our top four coefficents are .239, .116, .076, and .073. Lets see which stats these correspond to

In [106]:
most=np.argsort(np.abs(pipeline.named_steps['regressor'].coef_))[-4:]

for i in reversed([statsrel[features].columns[i] for i in most]):
    print(i)

K%
Soft%
LD%
Oppo%


Given our correlation analysis above and the success of xFIP and SIERA it is unsurprising that K% and K/BB are among the strongest predictors of future performance. Interestingly, a  O-Contact% (the rate at which a batter makes contact on pitches outside the strike zone against a given pitcher) has a higher coefficient than we we would expect. 